# STL Decomposition Algorithem 
## Introduction
STL(Seasonal and Trend Decompostion using LOESS) method was proposed by Cleveland et al. in 1990 for robust seasonal-trend decomposition proccedure based on LOESS smoothing.
It can be only applied on additive model which can be written as  
$Y_{\nu} = T_{\nu} + S_{\nu} + R_{\nu}$ , $\nu = 1, 2, ..., N$  
where $Y_{\nu}$ is the time series data, $T_{\nu}$, $S_{\nu}$, and $R_{\nu}$ are the trend component, seasonal component, and residual component respectively.  

STL algorithem is done through two loops. In the outer loop, robustness weights are assigned to each data point depending on the size of the remainder. This allows for reducing or eliminating the transient, aberrant behavior in the data. The inner loop extract the trend and seasonal components with LOESS smooghting and interatively updates until they converges. 


## LOESS
One of the key for STL decomposition is LOESS(locally weighted regression) smoothing. First we consider the non-weighted linear regression. Suppose $x_i$ and $y_i$ for $i=1$ to $N$ are measurements of an independent and dependent variable, respectively. The general expression for data $y_i$ by linear regression curve as  

$y_i = g(x_i) + \epsilon_i$  
$g(x) = \theta_0 + \theta_1 x + \theta_2 x^2 + ... + \theta_d x^d$

Linear means the coefficient $\theta$ is linear. To find the $g(x)$, we need to minimize the cost function $J(\theta)=\frac{1}{2N} \sum_{k=1}^{n}[g(x_k)-y_k]^2$ to get the $\theta$ value. For locally weighted regression, a positive interger $q$ need to be chosen, the large $q$, means a long span smoothing. For $q\leq N$, the $q$ values of $x_i$ that are closest to $x$ are selected and each is weighted based on how far it is from $x$. We will give a weighted function on certain point $x_i$ for cost function such that

$J_i(\theta)=\frac{1}{2N} \sum_{k=1}^{n}v_i(x)[g(x_k)-y_k]^2$  
$v_i(x) = W(\frac{|x_i - x|}{\lambda_q(x)}) $  

where $\lambda_q(x)$ be the distance fo the $q$th farthest $x_i$ from x. The $W(u)$ is the tri-cube funciont 

$W(u) =(1-u^3)^2$  for  $0\leq u \leq 1 $  
$W(u) = 0$  for  $u > 1 $ 

The weight function $W(u)$ is with the following properties  
* $W(x) > 0$ for $|x| < 1$  
* $W(-x) = W(x)$  
* $W(x)$ is a nonicreasing function for $x\geq 1$
* $W(x)=0$ for $|x| \geq 1$

The next step is to fit a polynomial of degree $d$ to the data with wight $v_i(x)$ at $(x_i, y_i)$. The value of the locally-fitted polynomial at x is $g(x)$. Therefore, we will get a lot of $g(x)$ depending how many data points you want to fit. Degree $d$ equal to $1$, $2$ or $0$, means the fitting is locally-linear, locally-quadratic or locally-horizontal line.

The LOESS regressoin curve $g(x)$ is a smoothing of $y$ given $x$ that can be computed for any value $x$ along the scale of indenpent variable. That is, LOESS is defined everywhere  and not just at dataset $x_i$. This is an important feature that in STL will allow us to deal with missing values and detrend the seasonal component in a straightforward way.

Now suppose that $q>N$, $\lambda_n(x)$ is the distance from $x$ to the farthest $x_i$. For $q>N$, we defined $\lambda_q(x)$ by  
$\lambda_q(x) = \lambda_n(x)\frac{q}{n}$


## Inner Loop

Inner loop contain 6 steps, the number of loop is $n_i$, iteratively calculate trend and seasonal terms until converge. 

#### **Step 1：*Detrending***
$Y_{\nu} - T_{\nu}^k$ 
where $k = 0, 1, ..., n_i$ is the loop number. If the observed value $Y_{\nu}$ is missing, then the detrended term is also missing.

Initial conditions：$T_{\nu}^k=0$ and $R_{\nu}^k=0$

#### **Step 2：*Cycle-subseries Smoothing***
The detrended time series is broken into cycle-subseries. For example, monthly data with a periodicity of 12 months would yield 12 cycle-subseries, rirst cycle-suberies will only include all Jan. data. Each cycle-subseries is then LOESS smoothed with data with $q=n_s$ and $d_s=1$. Smoothed values are computed at all time positions of the cycle-suberies, including those with missing value. Here we also need to extend cycle-subseries data at the beginning position and the end position. For example, if the Jan. cycle-subseries ranges from 1943 to 1985 with a missing data at 1960, then the smoothed values are computed at all postions from 1942 ot 1986. The purpose of this extending tricky part will be revealed at step3. Finally, the smoothed values yield a temporary seasonal series $C_{\nu}^{(k+1)}$, $\nu=-n_p, ..., 1, 0, 1, ..., N+n_p$, total count of $C_{\nu}^{(k+1)}$ is $N+2n_p$

#### **Step 3：*Low-Pass Filtering of Smoothed Cycle-Subseries***
Applied $n_p$, $n_p$, $3$ centered moving average on $C_{\nu}^{(k+1)}$ then LOESS smoothed with $q=n_l$ and $d_l=1$. The output is defined as $L_{\nu}^{(k+1)}$, where $\nu$ is $1$ to $N$ because three centered moving averages will reducee the output count. That's why we need to extend data points in step2 to avoid this loss. The purpose step3 is to identify any trend-cycle that may have contaminated $C_{\nu}^{(k+1)}$ in step2.

#### **Step 4：*Detrending of Smoothed Cycle-Subseries***
$S_{\nu}^{(k+1)} = C_{\nu}^{(k+1)} - L_{\nu}^{(k+1)}$. 
This is the seasonal component from the (k+1)th loop for $\nu=1, 2, ..., N$

#### **Step 5：*Deseasonalizing***
$Y_{\nu} - S_{\nu}^{(k+1)}$. If $Y_{\nu}$ is missing at particular time position, the deseasonalized sereis is also missing.

#### **Step 6：*Trend Smoothing***
Smoothed deseasonalized series $Y_{\nu} - S_{\nu}^{(k+1)}$ by LOESS with $q=n_t$ and $d_t=1$, even for those missing values. Then we get $T_{\nu}^{k+1}$ for $\nu=1, 2, ..., N$.

Then iterate $n_i$ times from step1 to step6 to get the final result, we can also use the convergence criterion to check whether the iterations are enough or not.  

#### **Convergence Criterion**
In the original paper, auther defined $U_{\nu}^k$ and $U_{\nu}^{(k+1)}$ are successive iterates of either a trend or seasonal component, then $U_{\nu}^k$ was judged to have been converged component if
$$ \frac{max |U_{\nu}^k - U_{\nu}^{(k+1)}|}{max(U_{\nu}^k) - min(U_{\nu}^k)} < 0.01 $$
Surely, we can choose our own convergence criterion.

#### **Result**
After checking the output is convergence, then we can get  

$S_{\nu} = S_{\nu}^{(k+1)}$  
$T_{\nu} = T_{\nu}^{(k+1)}$  
$R_{\nu} = Y_{\nu} - S_{\nu} - T_{\nu}$ 

Unlike $T_{\nu}$ and $S_{\nu}$, $R_{\nu}$ is not defined where $Y_{\nu}$ has missing values.

## Outer Loop
Supposed we carried out an initial run of the inner loop to get estimates, $T_{\nu}$ and $S_{\nu}$, then residual is $R_{\nu} = Y_{\nu} - S_{\nu} - T_{\nu}$. The robust weights reflect how extreme $R_\nu$ is, it is defined as

* robustness weight $\rho_{\nu}=B(\frac{|R_{\nu}|}{h})$, $B(u)$ is called bi-square wight function
* h = 6 * median( $|R_{\nu}|$ )
* $B(u) = (1-u^2)^2$ for $0\leq u \leq 1 $
* $B(u) = 0$ for $u > 1 $ 

Now we can repeat inner loop, but the **Step2** and **6**, the nighborhood weight for a value at time $\nu$ is multiplied by the robustness weight, which become $v_\nu\rho_\nu$. These robustness iterations of the outer loop is $n_0$ times.

## Pros and Cons of STL
#### **Pros**
* Flexibility in specifying the amounts of variation in the trend and seasonal components
* Capable of handling seasonal time series where the length of seasonality is other than quarterly or monthly. In fact, any seasonal period $n_p>1$ is allowed.
* Able to decompose the missing vaule.
* Robust trend and seasonal compoents that are not distorted by transient, aberrant behavior in the data, So occasional unusual observations will not affect the estimates of the trend-cycle and seasonal components. They will, however, affect the remainder component.
* Easy computer implementation and fast computation, even for long time series.

#### **Cons**
* Does not automatically handle trading day or calendar variation.
* Only provides facilities for additive decompositions.


## Choosing STL Parameters  
> ***Parameters corresponding to R stl Function***  
    > $n_i$(inner): inner loop iterations  
    > $n_o$(outer): outer loop iterations  
    > $n_p$(?)  
    > $n_s$(s.window): the span of the loess window for seasonal extraction, which should be odd and at least 7.   
    > $n_l$(l.window): the span of the loess window of the low-pass filter, should be odd number.    
    > $n_t$(t.window): the span of the loess window for trend extraction, which should be odd.  
    > $d_s$(s.degree): degree of locally-fitted polynomial in seasonal extraction. Should be 0 or 1. Default = 0.  
    > $d_l$(l.degree): degree of locally-fitted polynomial for the low-pass filter. Must be 0 or 1. Default = t.degree  
    > $d_t$(t.degree): degree of locally-fitted polynomial in trend extraction. Should be 0 or 1. Default = 1  
    > roubst = TURE or FALSE  
    > $n_s^{jump}$(s.jump): default is ceiling(s.window/10)  
    > $n_l^{jump}$(l.jump): default is ceiling(l.window/10)    
    > $n_t^{jump}$(t.jump):default is ceiling(t.window/10)
    
Author claim that the convergence is very fast without outer loop, in many cases, $n_i=1$ is sufficient, but $n_i=2$ is recommended to provide near certainty of convergence. 
Suppose now that we need robustness iterations, taking $n_i=1$ and large $n_o$ so that the robust estimates of the trend and seasonal components converge. In R stl function default setting  
$n_i = if(robust=TRUE)$ $1$ $else$ $2$  
$n_o = if(robust=TRUE)$ $15$ $else$ $0$

$d_s$, $d_l$, and $d_t$ are the degree of the loess smoothing, which typcially be choosen as $1$, that means the fitting is locally-linear.  

The trend smoothing is carried out at positions $1$, $1+ n_t^{jump}$, $1+2n_t^{jump}$, and so forth, and at postion $N$. The ohter position is computed by linear interpolation. A similiar procedure with the parameter $n_s^{jump}$ and $n_l^{jump}$. Author found that taking $n_t^{jump}$ to be the smallest integer greater than or equal to $n_t/10$ or even $n_t/5$ works quite well. Similar statements hold for $n_s^{jump}$ and $n_l^{jump}$.

## References
* [R. B. Cleveland, W. S. Cleveland, J.E. McRae, and I. Terpenning (1990) STL: A Seasonal-Trend Decomposition Procedure Based on Loess. Journal of Official Statistics, 6, 3--73](https://www.wessa.net/download/stl.pdf)
* [William S. Cleveland(1979) Robust Locally Weighted Regression and Smoothing Scatterplots](https://www.google.com.tw/url?sa=t&rct=j&q=&esrc=s&source=web&cd=1&cad=rja&uact=8&ved=0ahUKEwi4vNr-ievXAhURNrwKHe8_B4kQFggpMAA&url=http%3A%2F%2Fwww.stat.washington.edu%2Fcourses%2Fstat527%2Fs13%2Freadings%2FCleveland_JASA_1979.pdf&usg=AOvVaw20WQwWueh5hfbv3L85uPju)
* [STL Algorithm Explained: STL Part II](http://www.gardner.fyi/blog/STL-Part-II/)
* [R Documentation：stl](https://www.rdocumentation.org/packages/stats/versions/3.4.1/topics/stl)
* [Otext STL Decomposition](https://www.otexts.org/fpp/6/5)
* [Engineering Statistics：LOESS](http://www.itl.nist.gov/div898/handbook/pmd/section1/pmd144.htm)
* [Locally weighted regression](https://medium.com/100-days-of-algorithms/day-97-locally-weighted-regression-c9cfaff087fb)
* [Python-wrapper for R's STL](https://gist.github.com/andreas-h/7808564)



## Question
1. When we calculate the g(x) in the end of dataset by LOESS, it looks like we only fit the curve by lefthand side data, since righthand side is no $y_i$ for us to calculate cost function. Is this statement correct?  
* Low-pass filter in Step3 of inner loop is simple moving average or centered moving average? I only see moving average in original paper.
* How dose stl funtion in R to determine the periodicity of the seasonality? I don't find $n_p$ in the arguments.  
* Is there a guidline for how to choose $n_s$, $n_l$, $n_t$?  
